In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import catboost
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import Normalizer
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train["target"] = df_train.target.astype("category")

In [ ]:
df_train_columns = list(df_train)
for i in range(40, 50):
    df_train.hist(column="var_"+str(i))

In [ ]:
num_train = df_train.select_dtypes(include=["number"])
cat_train = df_train.select_dtypes(exclude=["number"])

Q1 = num_train.quantile(0.25)
Q3 = num_train.quantile(0.75)
IQR = Q3 - Q1

idx = ~((num_train < (Q1 - 1.5 * IQR)) | (num_train > (Q3 + 1.5 * IQR))).any(axis=1)
df_train_cleaned = pd.concat([num_train.loc[idx], cat_train.loc[idx]], axis=1)

In [ ]:
df_train_cleaned.info()

In [ ]:
df_train_cleaned.target.value_counts()

In [ ]:
df_train.head()

In [ ]:
df_train_cleaned.set_index("ID_code", inplace=True)

In [ ]:
X = df_train_cleaned.drop("target", axis=1)
y = df_train_cleaned["target"]

In [ ]:
X.head()

In [ ]:
X = (X - X.mean()) / (X.max() - X.min())

In [ ]:
X.head()

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, random_state=1, test_size=0.2)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


In [ ]:
for train_index, val_index in sss.split(X_train, y_train):
    X_train, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

In [ ]:
#X_train.set_index("ID_code", inplace=True)
#X_test.set_index("ID_code", inplace=True)
#X_val.set_index("ID_code", inplace=True)

In [ ]:
from catboost import CatBoostClassifier, Pool, cv
model = CatBoostClassifier(
    eval_metric="WKappa",
    random_seed=42,
    logging_level="Silent",
    use_best_model=True
)

model.fit(X_train, y_train,
         eval_set=(X_val, y_val),
         plot=True
         )

In [ ]:
predicted = model.predict(X_test)
score = roc_auc_score(y_score=predicted, y_true=y_test)
print(score)

In [ ]:
def roc_score(y_score, y_true):
    return tf.py_function(roc_auc_score, (y_score, y_true), tf.double)

def f1_keras(y_score, y_true):
    return tf.py_function(f1_score, (y_score, y_true), tf.double)

keras_model = Sequential()
keras_model.add(Dense(4, activation="relu", input_dim=200))
keras_model.add(Dense(4, activation="relu"))
keras_model.add(Dense(1, activation="sigmoid"))

keras_model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"])
keras_model.summary()

In [ ]:
keras_model.fit(X_train, y_train, epochs=100, batch_size=200)
score = keras_model.evaluate(X_test, y_test, batch_size=200)
predicted = keras_model.predict(X_test)
score = roc_auc_score(y_score=predicted, y_true=y_test)
print(score)

In [ ]:
df_train.set_index("ID_code", inplace=True)
X = df_train.drop("target", axis=1)
y = df_train["target"]
X = (X - X.mean()) / (X.max() - X.min())
sss = StratifiedShuffleSplit(n_splits=1, random_state=1, test_size=0.2)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
for train_index, val_index in sss.split(X_train, y_train):
    X_train, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
keras_model = Sequential()
keras_model.add(Dense(4, activation="relu", input_dim=200))
keras_model.add(Dense(4, activation="relu"))
keras_model.add(Dense(1, activation="sigmoid"))

keras_model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"])
keras_model.summary()
keras_model.fit(X_train, y_train, epochs=100, batch_size=200)
score = keras_model.evaluate(X_test, y_test, batch_size=200)
predicted = keras_model.predict(X_test)
score = roc_auc_score(y_score=predicted, y_true=y_test)
print(score)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, auc
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l1_l2, l1
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Read in the data
full_train_set = pd.read_csv('../input/train.csv')
submission_test_set = pd.read_csv('../input/test.csv')

features = [x for x in full_train_set.columns if x.startswith("var")]

hist_df = pd.DataFrame()
for var in features:
    var_stats = full_train_set[var].append(submission_test_set[var]).value_counts()
    hist_df[var] = pd.Series(submission_test_set[var]).map(var_stats)
    hist_df[var] = hist_df[var] > 1

ind = hist_df.sum(axis=1) != 200
var_stats = {var:full_train_set[var].append(submission_test_set[ind][var]).value_counts() for var in features}

def make_model_new():
    model = Sequential()
    model.add(Dense(200, input_dim=2, activation='relu', 
              kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu', kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.15))
    model.add(Dense(25, activation='relu', kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

from scipy.special import logit
pred = 0
counter = 1
for var in features:
    print("Feature %i" % counter)
    model = make_model_new()
    model.fit(np.hstack([full_train_set[var].values.reshape(-1,1),
                                 full_train_set[var].map(var_stats[var]).values.reshape(-1,1)]),
                               full_train_set["target"].values, verbose=2, batch_size=1000, epochs=10)
    pred += logit(model.predict(np.hstack([submission_test_set[var].values.reshape(-1,1),
                                 submission_test_set[var].map(var_stats[var]).values.reshape(-1,1)])))
    counter += 1